In [1]:
using BenchmarkTools

soft_threashold_bc(x, λ) = @. sign(x) * max(0, abs(x) - λ)

function soft_threashold_naive(x, λ)
    y = zero(x)
    ind = findall(>(λ), x)
    y[ind] = x[ind] .- λ
    ind = findall(<(-λ),  x)
    y[ind] = x[ind] .+ λ
    y
end

x20 = randn(20)
@show soft_threashold_bc(x20, 1.0) == soft_threashold_naive(x20, 1.0)
print("soft_threashold_bc(x20, 1.0):    "); @btime soft_threashold_bc($x20, 1.0)
print("soft_threashold_naive(x20, 1.0) "); @btime soft_threashold_naive($x20, 1.0)
println()

x1000 = randn(1000)
@show soft_threashold_bc(x1000, 1.0) == soft_threashold_naive(x1000, 1.0)
print("soft_threashold_bc(x1000, 1.0):     "); @btime soft_threashold_bc($x1000, 1.0)
print("soft_threashold_naive(x1000, 1.0) "); @btime soft_threashold_naive($x1000, 1.0)
;

soft_threashold_bc(x20, 1.0) == soft_threashold_naive(x20, 1.0) = true
soft_threashold_bc(x20, 1.0):      33.502 ns (2 allocations: 224 bytes)
soft_threashold_naive(x20, 1.0)   271.885 ns (20 allocations: 896 bytes)

soft_threashold_bc(x1000, 1.0) == soft_threashold_naive(x1000, 1.0) = true
soft_threashold_bc(x1000, 1.0):       518.966 ns (3 allocations: 7.88 KiB)
soft_threashold_naive(x1000, 1.0)   1.370 μs (21 allocations: 15.58 KiB)


In [2]:
soft_threashold_bc!(y, x, λ) = @. y = sign(x) * max(0, abs(x) - λ)

y1000 = similar(x1000)
@show soft_threashold_bc(x1000, 1.0) == soft_threashold_bc!(y1000, x1000, 1.0)
print("soft_threashold_bc(x1000, 1.0):         "); @btime soft_threashold_bc($x1000, 1.0)
print("soft_threashold_bc!(y1000, x1000, 1.0): "); @btime soft_threashold_bc!($y1000, $x1000, 1.0);

soft_threashold_bc(x1000, 1.0) == soft_threashold_bc!(y1000, x1000, 1.0) = true
soft_threashold_bc(x1000, 1.0):           554.217 ns (3 allocations: 7.88 KiB)
soft_threashold_bc!(y1000, x1000, 1.0):   362.981 ns (0 allocations: 0 bytes)


In [3]:
soft_threashold_bc2!(y, x, λ) = @. y = (x > λ) * (x - λ) + (x < -λ) * (x + λ)

y1000 = similar(x1000)
z1000 = similar(x1000)
@show soft_threashold_bc!(y1000, x1000, 1.0) == soft_threashold_bc2!(z1000, x1000, 1.0)
print("soft_threashold_bc(x1000, 1.0):          "); @btime soft_threashold_bc($x1000, 1.0)
print("soft_threashold_bc!(y1000, x1000, 1.0):  "); @btime soft_threashold_bc!($y1000, $x1000, 1.0)
print("soft_threashold_bc2!(z1000, x1000, 1.0): "); @btime soft_threashold_bc2!($z1000, $x1000, 1.0);

soft_threashold_bc!(y1000, x1000, 1.0) == soft_threashold_bc2!(z1000, x1000, 1.0) = true
soft_threashold_bc(x1000, 1.0):            547.273 ns (3 allocations: 7.88 KiB)
soft_threashold_bc!(y1000, x1000, 1.0):    367.943 ns (0 allocations: 0 bytes)
soft_threashold_bc2!(z1000, x1000, 1.0):   293.385 ns (0 allocations: 0 bytes)


In [4]:
@show findall(>(1.0), x1000) |> typeof
@show (x1000 .> 1.0) |> typeof
println()

function soft_threashold_naive2(x, λ)
    y = zero(x)
    ind = x .> λ
    y[ind] = x[ind] .- λ
    ind = x .< -λ
    y[ind] = x[ind] .+ λ
    y
end

@show soft_threashold_naive(x1000, 1.0) == soft_threashold_naive2(x1000, 1.0)
print("soft_threashold_naive(x1000, 1.0):  "); @btime soft_threashold_naive($x1000, 1.0)
print("soft_threashold_naive2(x1000, 1.0): "); @btime soft_threashold_naive2($x1000, 1.0)
;

findall((>)(1.0), x1000) |> typeof = Vector{Int64}
(x1000 .> 1.0) |> typeof = BitVector

soft_threashold_naive(x1000, 1.0) == soft_threashold_naive2(x1000, 1.0) = true
soft_threashold_naive(x1000, 1.0):    1.400 μs (21 allocations: 15.58 KiB)
soft_threashold_naive2(x1000, 1.0):   1.120 μs (17 allocations: 13.16 KiB)


In [5]:
versioninfo()

Julia Version 1.11.0
Commit 501a4f25c2 (2024-10-07 11:40 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 12 × Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LLVM: libLLVM-16.0.6 (ORCJIT, skylake)
Threads: 12 default, 0 interactive, 6 GC (on 12 virtual cores)
Environment:
  JULIA_DEPOT_PATH = D:\.julia
  JULIA_NUM_PRECOMPILE_TASKS = 4
  JULIA_NUM_THREADS = 12
  JULIA_PYTHONCALL_EXE = D:\.julia\conda\3\python.exe
